In [107]:
import numpy as np
import pandas as pd
import scipy.io

import warnings
warnings.filterwarnings('ignore')
import gc
gc.collect()

460

In [108]:
!pip install pretrainedmodels
!pip install utils

%reload_ext autoreload
%autoreload 2
%matplotlib inline

!pip install fastai==1.0.57

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [109]:
#fastai
# import fastai

# from fastai import *
# from fastai.vision import *

# from torchvision.models import *
# import pretrainedmodels

# from utils import *
# import sys

# from fastai.callbacks.hooks import *

# from fastai.callbacks.tracker import EarlyStoppingCallback
# from fastai.callbacks.tracker import SaveModelCallback

In [110]:
#load data
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/Shareddrives/Topgun/dataset/train

# !unzip -qq "/content/drive/Shareddrives/Topgun/apple_data.zip"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Shareddrives/Topgun/dataset/train


In [111]:
from glob import glob
filepaths = list(glob('*.JPG'))
len(filepaths)

1200

In [112]:
#load csv file
csv_path = "/content/drive/Shareddrives/Topgun/Apple.csv"
df = pd.read_csv(csv_path)
# print(df)


In [113]:
import os
from glob import glob
import pandas as pd   

In [114]:
df = pd.read_csv('/content/drive/Shareddrives/Topgun/Apple.csv', encoding='utf-8')
sugar = df['sugar']

In [115]:
data_dict = { 'apple': [], 'imageName':[], 'sugar':[], 'file_path':[]}
   
for i in range(200):     #0-159
  for j in range(1, 7):
    filename = str(i+1)+'-'+str(j)+'.JPG'
    img_path = '/content/drive/Shareddrives/Topgun/dataset/train'+'/'+filename

    data_dict['apple'].append(i+1)
    data_dict['imageName'].append(filename)
    data_dict['sugar'].append(sugar[i])
    data_dict['file_path'].append(img_path) # file_path 항목에 파일 경로 저장

    
  
apple_df = pd.DataFrame(data_dict)
# print('\n<data frame>\n', apple_df)

apple_df.to_csv("topgun.csv", mode='w')

In [116]:
# 이미지 데이터셋 다운로드
import os
import torch
from torch.utils.data import Dataset
from skimage import io
from PIL import Image, UnidentifiedImageError,ImageFile

In [117]:
csv = pd.read_csv('/content/drive/Shareddrives/Topgun/dataset/topgun.csv')

*Custom Dataset*

In [118]:
class AppleImageDataset(Dataset):
  def __init__(self, csv_file, root_dir):
    super(AppleImageDataset, self).__init__()
    self.annotations = pd.read_csv(csv_file)
    self.root_dir = root_dir

  def __len__(self):
      return len(self.annotations) #1200 img
    
  def __getitem__(self, index):
      #apple number
      apple = int(self.annotations.iloc[index, 1]) #[0]

      #sugar level
      sugar = float(self.annotations.iloc[index, 3])  #[1]

      #img path
      image_path = self.annotations.iloc[index, 4] #[2]

      # PIL.Image로 이미지 로드
      image = Image.open(image_path).convert('RGB') #[3]
      #   # transform 적용
      # img = self.transform(img)

      return (image_path, sugar, apple, image)


In [119]:
AppleDataset = AppleImageDataset(csv_file = '/content/drive/Shareddrives/Topgun/dataset/topgun.csv', root_dir = '/content/drive/Shareddrives/Topgun/dataset/train')

Split into train & validation set

In [120]:
import random

appleTotal = int(len(AppleDataset)/6) # Apple list
appleList = list(range(1, appleTotal)) # Apple image list

random.seed(310)  # fix the seed so the shuffle will be the same everytime
random.shuffle(appleList)

train_size = int(appleTotal*0.8) #160

#train set
indicesT = list(range((int(appleList[0])-1)*6, int(appleList[0])*6))

for i in range(1, train_size):
  appleID = int(appleList[i])
  indicesT.extend(list(range((appleID-1)*6, appleID*6)))
  
train_dataset_split = torch.utils.data.Subset(AppleDataset, indicesT)

#validation set
indicesV = list(range((int(appleList[train_size])-1)*6, int(appleList[train_size])*6))

for i in range(train_size+1, appleTotal-1):
  appleID = int(appleList[i])
  indicesV.extend(list(range((appleID-1)*6, appleID*6)))
  
val_dataset_split = torch.utils.data.Subset(AppleDataset, indicesV)


DataLoader 